In [1]:
import cv2
from skimage.filters import threshold_otsu
import numpy as np
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                                white_tophat)
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from skimage.draw import rectangle
%matplotlib inline 

In [2]:
def contour_imgs(img,contours,area):
    i=0
    big=0
    for contour in contours:
        x = contour[:,1]
        y = contour[:,0]
        [Xmin, Xmax, Ymin, Ymax] = [np.amin(x), np.amax(x), np.amin(y), np.amax(y)]
        Ymin = int(Ymin)
        Ymax = int(Ymax)
        Xmax = int(Xmax)
        Xmin = int(Xmin)
        c = img[Ymin:Ymax,Xmin:Xmax]
        if(area/20 < (Ymax-Ymin)*(Xmax-Xmin)):
            big+=1
        cv2.imwrite("contours/"+str(i)+".png",c)
        i+=1
    return big


In [147]:
im_in = cv2.imread("er4.png", cv2.IMREAD_GRAYSCALE)
th, im_th = cv2.threshold(im_in, 220, 255, cv2.THRESH_BINARY_INV)
###################Remove words###############
# structureElm =  np.array((
# 	[1,1,1],
# 	[1,1,1]), dtype="int")
# closed = closing(im_th, structureElm)

# for i in range(30):
#     closed = closing(closed, structureElm)

# cv2.imwrite("closed.png", closed)

# im2, contours_cv, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# for cnt in contours_cv:
#     x,y,w,h = cv2.boundingRect(cnt)
#     if w>5 and w<300 and h>5 and h<30 :
#         rr, cc = rectangle(start = (y,x), end = (y+h,x+w), shape=im_in.shape)
#         im_th[rr.astype(int), cc.astype(int)] =  0


In [148]:
# Mask used to flood filling.
# Notice the size needs to be 2 pixels than the image.
h, w = im_th.shape[:2]
area = h*w
mask = np.zeros((h+2, w+2), np.uint8)

im_floodfill = im_th.copy()
##################Flood fill corners#####
cv2.floodFill(im_floodfill, mask, (0,0), 255)
cv2.floodFill(im_floodfill, mask, (w-1,0), 255)
cv2.floodFill(im_floodfill, mask, (0,h-1), 255)
cv2.floodFill(im_floodfill, mask, (w-1,h-1), 255)
#cv2.floodFill(im_floodfill, mask, (int(w/2),int(h/2)), 255)
# cv2.floodFill(im_floodfill, mask, (int(w*0.75),int(h/2)), 255) 


(0,
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]], dtype=uint8),
 (0, 0, 0, 0))

In [167]:
######################cv contours#########
# Invert floodfilled image
im_floodfill_inv = cv2.bitwise_not(im_floodfill)
im2, contours_cv, hierarchy = cv2.findContours(im_floodfill_inv, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
im_empty = np.ones((h, w,3), np.uint8) * 255

i=0
for cnt in contours_cv:
    x,y,w1,h1 = cv2.boundingRect(cnt)
    contour_area = w1*h1
    area_per = contour_area/area
    # if(i==8):
    im_temp = np.ones((h, w,3), np.uint8) * 255
    cv2.drawContours(im_temp, contours_cv, i, (0,0,0), 1)
    gray_img=cv2.cvtColor(im_temp,cv2.COLOR_BGR2GRAY) 
    gray_img = np.float32(gray_img)
    corners = None
    corners = cv2.goodFeaturesToTrack(gray_img, 0, 0.1, 20,useHarrisDetector=True)
    #print(type(corners))
    if(corners is None):
        i+=1
       # print("hi")
        continue
    corners = np.int0(corners)
    cornersCount = 0
    for c in corners:
        xc, yc = c.ravel()
        if((xc-x)/w <= 0.1 or (x+w-xc)/w <=0.1 or (yc-y)/h <=0.1 or (y+h-yc)/h <=0.1):
            cornersCount +=1
            cv2.circle(im_empty, (xc, yc), 1, 255, -1)
    #print(i,len(corners))
    if(cornersCount>5):
        i+=1
        continue
    cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
    # cv2.imwrite("gray.png", gray_img)
        # break
    i+=1
#cv2.drawContours(im, contours_cv, -1, (0,0,0), 1)
cv2.imwrite("contoured.png", im_empty)
#########################################
# Display images.
cv2.imwrite("Thresholded_Image.png", im_th)
cv2.imwrite("Floodfilled_Image.png", im_floodfill)
cv2.imwrite("Inverted_Floodfilled_Image.png", im_floodfill_inv)
#start flooding from corners?

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 2
28 1
29 1
30 1
31 1
32 1
33 1
34 1
35 1
36 1
37 1
38 1
39 1
40 1
41 1
42 1
43 4
44 1
45 1
46 1
47 1
48 1
49 1
50 1
51 1
52 1
53 1
54 1
55 1
56 1
57 1
58 2
59 1
60 1
61 1
62 1
63 1
64 1
65 1
66 1
67 1
68 1
69 1
70 1
71 1
72 1
73 1
74 1
75 1
76 1
77 1
78 1
79 1
80 1
81 1
82 1
83 1
84 1
85 1
86 1
87 1
88 1
89 1
hi
91 6
92 6
93 1
94 2
95 1
96 1
97 1
98 1
99 1
100 1
101 1
102 2
103 1
104 1
105 1
106 1
107 1
108 1
109 1
110 1
111 1
112 1
113 4
114 1
115 1
116 1
117 1
118 1
119 1
120 1
121 1
122 1
123 1
124 1
125 1
126 2
127 2
128 1
129 1
130 1
131 1
132 1
133 2
134 1
135 1
136 1
137 1
138 1
139 1
140 1
141 1
142 1
143 1
144 1
145 1
146 3
147 1
148 6
149 1
150 1
151 1
152 1
153 1
154 1
155 1
156 1
157 1
158 1
159 2
160 1
161 1
162 1
163 1
164 4
165 1
166 1
167 1
168 1
169 1
170 1
171 1
172 1
173 1
174 1
175 1
176 1
177 1
178 4
179 1
180 1
181 1
182 1
183 1
184 1
18

True

In [133]:
kernel1 = np.array((
    [0,0,0],
	[-1,1,-1],
    [0,0,0]), dtype="int")
kernel2 = np.array((
	[0,-1,0],
    [0,1,0],
    [0,-1,0]), dtype="int")
kernel3 = np.array((
	[-1,0,0],
    [0,1,0],
    [0,0,-1]), dtype="int")
kernel4 = np.array((
	[0,0,-1],
    [0,1,0],
    [-1,0,0]), dtype="int")
kernel5 = np.array((
	[0,-1,0],
    [-1,4,-1],
    [0,-1,0]), dtype="int")
#only bar edges after floodfilling the image
image_with_edges = cv2.filter2D(im_floodfill, -1, kernel1)
image_with_edges += cv2.filter2D(im_floodfill, -1, kernel2)
image_with_edges += cv2.filter2D(im_floodfill, -1, kernel3)
image_with_edges += cv2.filter2D(im_floodfill, -1,kernel4)
#by subtraction we get all step edges
edges = cv2.filter2D(im_floodfill, -1, kernel5)

edges_ored = cv2.bitwise_not(image_with_edges & im_floodfill)
cv2.imwrite("edges.png", image_with_edges)
cv2.imwrite("edges2.png", edges)
cv2.imwrite("edges_ored.png", edges_ored)




True

In [ ]:
        corners = cv2.cornerHarris(gray_img,4,3,0.05)
        ret, corners = cv2.threshold(corners,0.02*corners.max(),255,0)
        corners = np.uint8(corners)
        # find centroids
        ret, labels, stats, centroids = cv2.connectedComponentsWithStats(corners)
        # define the criteria to stop and refine the corners
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
        corners = cv2.cornerSubPix(gray_img,np.float32(centroids),(5,5),(-1,-1),criteria)
        # Now draw them
        res = np.hstack((centroids,corners))
        res = np.int0(res)
        im_empty[res[:,1],res[:,0]]=[0,0,255]
        im_empty[res[:,3],res[:,2]] = [0,255,0]
        print(i,len(corners),res[:,1])

In [61]:
help(cv2.goodFeaturesToTrack)
#TODO
#instead of removing the words check for corners near border

Help on built-in function goodFeaturesToTrack:

goodFeaturesToTrack(...)
    goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance[, corners[, mask[, blockSize[, useHarrisDetector[, k]]]]]) -> corners
    .   @brief Determines strong corners on an image.
    .   
    .   The function finds the most prominent corners in the image or in the specified image region, as
    .   described in @cite Shi94
    .   
    .   -   Function calculates the corner quality measure at every source image pixel using the
    .   #cornerMinEigenVal or #cornerHarris .
    .   -   Function performs a non-maximum suppression (the local maximums in *3 x 3* neighborhood are
    .   retained).
    .   -   The corners with the minimal eigenvalue less than
    .   \f$\texttt{qualityLevel} \cdot \max_{x,y} qualityMeasureMap(x,y)\f$ are rejected.
    .   -   The remaining corners are sorted by the quality measure in the descending order.
    .   -   Function throws away each corner for which there is a st